In [ ]:
import google.colab
google.colab.auth.authenticate_user()

#authentication and establish connection
from google.colab import drive
drive.mount('/content/drive')

!ls '/content/drive/My Drive'

Mounted at /content/drive
 148.docx   BasedOnMaterial   ClassData		 Denomination   heart_disease_data.csv
 148.pdf    BasedOnPeriod    'Colab Notebooks'	 Edited.csv     models


In [ ]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.7 MB/s eta 0:00:00


**Web app to run all available models**

In [ ]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import numpy as np
import pandas as pd
from joblib import load
import PIL
import cv2
import base64

st.set_page_config(
    page_title="Digispectra App",
    layout="wide",
    initial_sidebar_state="expanded",
)


@st.cache_data
def get_img_as_base64(file):
    with open(file, "rb") as f:
        data = f.read()
    return base64.b64encode(data).decode()

img = get_img_as_base64("/content/drive/MyDrive/models/final.png")

page_bg_img = f""" <style>
                [data-testid="stAppViewContainer"] > .main {{
                background-image: url("https://i.postimg.cc/L8hx7hWk/Screenshot-2024-05-13-at-10-41-27-PM.png");
                background-size: 100%;
                background-position: top left;
                background-repeat: no-repeat;
                background-attachment: local;
                }}

                [data-testid="stSidebar"] > div:first-child {{
                background-image: url("data:image/png;base64,{img}");
                background-position: center;
                background-repeat: no-repeat;
                background-attachment: fixed;
                }}

                [data-testid="stHeader"] {{
                background: rgba(0,0,0,0);
                }}

                [data-testid="stToolbar"] {{
                right: 2rem;
                }}
                </style>
                """

st.markdown(page_bg_img, unsafe_allow_html=True)

model1_path = '/content/drive/MyDrive/models/materialModel.h5'
model2_path = '/content/drive/MyDrive/models/classcoinModel.h5'
model3_path = '/content/drive/MyDrive/models/periodModel.h5'
model4_path = '/content/drive/MyDrive/models/denomModel.h5'

model1 = tf.keras.models.load_model(model1_path)
model2 = tf.keras.models.load_model(model2_path)
model3= tf.keras.models.load_model(model3_path)
model4= tf.keras.models.load_model(model4_path)

class_names1 = ['Base Silver', 'Copper', 'Copper Alloy', 'Gold', 'Silver']
class_names2 = ['Coin', 'Not Coin']
class_names3= ['MEDIEVAL', 'ROMAN']
class_names4= ['Cut halfpenny', 'Nummus (AE 1 - AE 4)', 'Penny', 'Radiate (antoninianus)']


#random forest prediction
model= load('/content/drive/MyDrive/models/randomForest4.joblib')
input_df= pd.read_pickle('/content/drive/MyDrive/models/input_data.pkl')

class_broad_period= ['Roman', 'Medieval']

def predict_broad_period(input_data):
    temp_df= input_df.copy(deep= True)
    temp_df.reset_index(drop=True, inplace=True)
    temp_df.loc[0, input_data]= 1
    temp_df= temp_df.fillna(0)
    prediction= model.predict(temp_df)
    return prediction

#material classification
def preprocess_image1(image):
    img = image.resize((180, 180))  # Resize image to match model input size
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Create batch axis
    return img_array

def predict_image1(image):
    img_array = preprocess_image1(image)
    predictions = model1.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])
    predicted_class = class_names1[predicted_class_index]
    confidence = predictions[0][predicted_class_index]
    return predicted_class, confidence

#denomination
def predict_image4(image):
    img_array = preprocess_image1(image)
    predictions = model4.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])
    predicted_class = class_names4[predicted_class_index]
    confidence = predictions[0][predicted_class_index]
    return predicted_class, confidence

#coin and not coin
def preprocess_image2(image):
    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    resized_img = tf.image.resize(img, (256, 256)).numpy() / 255.0
    expanded_img = np.expand_dims(resized_img, axis=0)
    return expanded_img

def predict_image2(image):
    preprocessed_img = preprocess_image2(image)
    prediction = model2.predict(preprocessed_img)
    if prediction > 0.5:
        return "Not Coin", prediction
    else:
        return "Coin", prediction

#MEDIEVAL, ROMAN
def predict_image3(image):
    preprocessed_img = preprocess_image2(image)
    prediction = model3.predict(preprocessed_img)
    if prediction > 0.5:
        return "ROMAN", prediction
    else:
        return "MEDIEVAL", prediction

#streamlit app
def main():
    selected_page = st.sidebar.radio("", ["Home", "Navigate", "About"])

    if "Home" in selected_page:
        st.markdown(
            "<h1 style='text-align: center; font-family: Times New Roman, sans-serif;'>Digital Archaeology Toolkit</h1>",
            unsafe_allow_html=True,
        )
        st.markdown(
            "<h4 style='color: brown; text-align: center; font-family: Times New Roman, sans-serif'>An approach to artefact analysis</h4>",
            unsafe_allow_html=True,
        )
        st.markdown("<br>", unsafe_allow_html=True)
        st.markdown(
            "<h5>Modules available</h5>",
            unsafe_allow_html=True,
        )
        st.write(
            "- **Object classification:** Classify into available classes namely coin and not coin."
        )
        st.write(
            "- **Broad period prediction:** Classify into Roman and Medieval time periods based on input data."
        )
        st.write(
            "- **Period classification:** Classify into time period directly by image."
        )
        st.write(
            "- **Material classification:** Extract coin material, classes available are gold, silver, copper etc."
        )
        st.write(
            "- **Denomination classification:** Extract coin denomination, classes available are nummus, penny, half penny etc."
        )
        image_path = '/content/drive/MyDrive/models/arch.png'
        st.markdown("<br>", unsafe_allow_html=True)
        st.image(image_path, use_column_width=True)

    elif "Navigate" in selected_page:
        st.sidebar.title("Navigation")
        page = st.sidebar.selectbox("Go to", ["Broad Period Prediction", "Period Classification", "Object Classification", "Material Classification",
                                          "Denomination Classification"])
        if page == "Broad Period Prediction":
            st.title('Broad Period Prediction')

            #get user input
            material_input = st.text_input('Material:')
            ruler_input = st.text_input('Ruler Name:')
            mint_input = st.text_input('Mint Name:')
            denom_input = st.text_input('Denomination Name:')

            input_data= [material_input, ruler_input, mint_input, denom_input]

            if st.button('Predict'):
                prediction = predict_broad_period(input_data)
                st.write('Predicted broad period:', class_broad_period[prediction[0]])

        elif page == "Period Classification":
            st.title("Period Classification")
            uploaded_file= st.file_uploader("Please upload an image file", type=["jpg", "jpeg", "png"])

            if uploaded_file is not None:
                image = cv2.imdecode(np.fromstring(uploaded_file.read(), np.uint8), 1)
                st.image(uploaded_file, caption="Uploaded Image", use_column_width=True)
                predicted_class, confidence = predict_image3(image)
                st.subheader("Prediction Result:")
                st.write(f"Predicted class: {predicted_class}")

        elif page == "Object Classification":
            st.title("Object Classification")
            uploaded_file= st.file_uploader("Please upload an image file", type=["jpg", "jpeg", "png"])

            if uploaded_file is not None:
                image = cv2.imdecode(np.fromstring(uploaded_file.read(), np.uint8), 1)
                st.image(uploaded_file, caption="Uploaded Image", use_column_width=True)
                predicted_class, confidence = predict_image2(image)
                st.subheader("Prediction Result:")
                st.write(f"Predicted class: {predicted_class}")

        elif page == "Material Classification":
            st.title("Material Classification")
            uploaded_file = st.file_uploader("Please upload an image file", type=["jpg", "jpeg", "png"])

            if uploaded_file is not None:
                image = PIL.Image.open(uploaded_file)
                st.image(image, caption="Uploaded Image", use_column_width=True)
                predicted_class, confidence = predict_image1(image)
                st.subheader("Prediction Result:")
                st.write(f"Predicted class: {predicted_class}")
                st.write(f"Confidence: {confidence}")

        elif page == "Denomination Classification":
            st.title("Denomination Classification")
            uploaded_file = st.file_uploader("Please upload an image file", type=["jpg", "jpeg", "png"])

            if uploaded_file is not None:
                image = PIL.Image.open(uploaded_file)
                st.image(image, caption="Uploaded Image", use_column_width=True)
                predicted_class, confidence = predict_image4(image)
                st.subheader("Prediction Result:")
                st.write(f"Predicted class: {predicted_class}")
                st.write(f"Confidence: {confidence}")


    elif "About" in selected_page:
        st.markdown("<h3 style='text-align: center; font-family: Times New Roman, sans-serif;'>Contact Us</h1>",
                unsafe_allow_html=True)
        st.markdown("<br>", unsafe_allow_html=True)
        st.write("**Get in touch via email in case of queries or any help**")
        st.markdown("<br>", unsafe_allow_html=True)
        st.write("- Kirti Tomer: [2003010125@ipec.org.in](mailto:2003010125@ipec.org.in)")
        st.write("- Ishita Gupta:[2003010114@ipec.org.in](mailto:2003010114@ipec.org.in)")
        st.write("- Khushi Gole: [2003010123@ipec.org.in](mailto:2003010123@ipec.org.in)")
        st.markdown("---")
        st.write("Students of Dr. APJ Abdul Kalam Technical University, Lucknow 226031, Uttar Pradesh")


if __name__ == "__main__":
    main()

Writing app.py


In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 35.202.182.11


In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 2.382s
your url is: https://wild-pandas-agree.loca.lt
^C


In [ ]:
#custom web address app
import urllib.request
import subprocess

print("Password/Endpoint IP for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Endpoint IP for localtunnel is: 35.202.182.11


In [ ]:
subprocess.Popen(['streamlit', 'run', 'app3.py', '&>/content/logs.txt', '&'])

<Popen: returncode: None args: ['streamlit', 'run', 'app3.py', '&>/content/l...>

In [ ]:
custom_subdomain = "digispectra-web-app"
subprocess.run(['npx', 'localtunnel', '--port', '8501', '--subdomain', custom_subdomain])

CompletedProcess(args=['npx', 'localtunnel', '--port', '8501', '--subdomain', 'digispectra-web-app'], returncode=1)